In [37]:
from bs4 import BeautifulSoup
import re
import codecs
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import os

import pymorphy2 

In [39]:
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words

In [40]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [ ]:
output_txt = 'articles_output.txt'
total_sites = 28026
path = '../content/'

with tqdm_notebook(total=total_sites) as pbar:
    with open(output_txt, 'w') as out_f:
        out_f.write('{}\t{}\n'.format('doc_id', 'articles'))
        for doc_id in range(1,total_sites+1):
            with open(path + str(doc_id) + ".dat",'r') as in_f:       
                soup = BeautifulSoup(in_f, 'lxml')
                result = [i.text.lower() for i in soup.find_all('article')]
                result = [i for i in re.split(r'\W+', ' '.join(result)) if i]
                result = [re.sub(r'[^ёЁА-я]', '', i) for i in result]
                morth=pymorphy2.MorphAnalyzer()
                result = [morth.parse(word)[0].normal_form  for word in result if pos(word) not in functors_pos]
                result = [word for word in result if word not in russian_stopwords and len(word)>2]
                result = re.sub('\s+',' ', ' '.join(result))   

                out_f.write('{}\t{}\n'.format(doc_id, result))
                pbar.update()

In [45]:
df = pd.read_csv(output_txt, sep='\t', encoding='utf-8', lineterminator='\n')
df.head(9)

,doc_id,articles
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
5,6,NaN
6,7,NaN
7,8,NaN
8,9,число самый любимый лакомство особенно летний ...


In [ ]:
group_df = pd.read_csv('train_groups.csv', sep=',', encoding='utf-8', lineterminator='\n')
group_df = group_df[['doc_id', 'group_id']]
group_df.index = group_df.doc_id
group_df.drop('doc_id', inplace=True, axis=1)
group_df = group_df.sort_index()
df = pd.merge(nimg_df, group_df, on='doc_id', how='left')

group_df = pd.read_csv('test_groups.csv', sep=',', encoding='utf-8', lineterminator='\n')
group_df = group_df[['doc_id', 'group_id']]
group_df.index = group_df.doc_id
group_df.drop('doc_id', inplace=True, axis=1)
group_df = group_df.sort_index()
df = pd.merge(df, group_df, on='doc_id', how='left')

df['group_id'] = df.where(pd.notna(df.group_id_x), df.group_id_y, axis=0).group_id_x.astype(int)
df.drop(['group_id_x','group_id_y'], inplace=True, axis=1)